<h3>Tableau Project<h3>

I chose to do a quick analysis and data cleanse in python before using the data in Tableau. The reason for this choice was to easily deal with Null values, and allow for a deeper understanding of the data prior to using Tableau for visualizations

In [4]:
import pandas as pd

In [3]:
df = pd.read_csv('Data/airbnb.csv')
df.head()

,Host Id,Host Since,Name,Neighbourhood,Property Type,Review Scores Rating (bin),Room Type,Zipcode,Beds,Number of Records,Number Of Reviews,Price,Review Scores Rating
0,5162530,NaN,1 Bedroom in Prime Williamsburg,Brooklyn,Apartment,NaN,Entire home/apt,11249.0,1.0,1,0,145,NaN
1,33134899,NaN,"Sunny, Private room in Bushwick",Brooklyn,Apartment,NaN,Private room,11206.0,1.0,1,1,37,NaN
2,39608626,NaN,Sunny Room in Harlem,Manhattan,Apartment,NaN,Private room,10032.0,1.0,1,1,28,NaN
3,500,2008-06-26,Gorgeous 1 BR with Private Balcony,Manhattan,Apartment,NaN,Entire home/apt,10024.0,3.0,1,0,199,NaN
4,500,2008-06-26,Trendy Times Square Loft,Manhattan,Apartment,95.0,Private room,10036.0,3.0,1,39,549,96.0


In [46]:
#check the columns
df.columns

#noticed extra whitespace on Neighbourhood column so I removed it
df['Neighbourhood'] = df['Neighbourhood '].str.strip()

In [54]:
#check the data types
df.dtypes

Host Id                         int64
Host Since                     object
Name                           object
Neighbourhood                  object
Property Type                  object
Review Scores Rating (bin)    float64
Room Type                      object
Zipcode                       float64
Beds                          float64
Number of Records               int64
Number Of Reviews               int64
Price                          object
Review Scores Rating          float64
Neighbourhood                  object
dtype: object

<h5>Clean the Data<h5>

In [34]:
df.describe()

,Host Id,Review Scores Rating (bin),Zipcode,Beds,Number of Records,Number Of Reviews,Review Scores Rating
count,3.047800e+04,22664.000000,30344.000000,30393.000000,30478.0,30478.000000,22664.000000
mean,1.273171e+07,90.738659,10584.854831,1.530089,1.0,12.018735,91.993230
std,1.190270e+07,8.957205,921.299397,1.015359,0.0,21.980703,8.750421
min,5.000000e+02,20.000000,1003.000000,0.000000,1.0,0.000000,20.000000
25%,2.701298e+06,85.000000,10017.000000,1.000000,1.0,0.000000,89.000000
50%,8.551693e+06,90.000000,10065.000000,1.000000,1.0,3.000000,93.000000
75%,2.120617e+07,95.000000,11216.000000,2.000000,1.0,13.000000,99.000000
max,4.303307e+07,100.000000,99135.000000,16.000000,1.0,257.000000,100.000000


In [36]:
NaN_count = df[df['Number Of Reviews'] > 0].isna().sum()

#use imputation to put the Mean review score in 'Number of Reviews' and 'Number of Reviews' where there is > 0 reviews
mean_review_bin = df[df['Number Of Reviews'] > 0]['Review Scores Rating (bin)'].mean()
mean_review = df[df['Number Of Reviews'] > 0]['Review Scores Rating'].mean()

#impute the data into the NaN columns
df.loc[df['Number Of Reviews'] > 0, 'Review Scores Rating'] = df.loc[df['Number Of Reviews'] > 0, 'Review Scores Rating'].fillna(mean_review)
df.loc[df['Number Of Reviews'] > 0, 'Review Scores Rating (bin)'] = df.loc[df['Number Of Reviews'] > 0, 'Review Scores Rating (bin)'].fillna(mean_review_bin)


In [37]:
#recheck the count
print(NaN_count)

Host Id                         0
Host Since                      2
Name                            0
Neighbourhood                   0
Property Type                   2
Review Scores Rating (bin)      0
Room Type                       0
Zipcode                       104
Beds                           46
Number of Records               0
Number Of Reviews               0
Price                           0
Review Scores Rating            0
dtype: int64


Clean the Zipcode data by adding the correct zipcode if its missing

In [51]:
#check the number of neighbourhoods
df['Neighbourhood'].value_counts()

Neighbourhood
Manhattan        16033
Brooklyn         11675
Queens            2278
Bronx              345
Staten Island      147
Name: count, dtype: int64

In [65]:
#check the data for manhattan neighbourhood places with no zipcode
df.loc[(df['Neighbourhood'] == 'Manhattan') & (df['Zipcode'].isna())]

,Host Id,Host Since,Name,Neighbourhood,Property Type,Review Scores Rating (bin),Room Type,Zipcode,Beds,Number of Records,Number Of Reviews,Price,Review Scores Rating,Neighbourhood
133,13654,2009-04-16,Room with a view in Central Harlem,Manhattan,Apartment,85.000000,Private room,NaN,1.0,1,11,75,85.00000,Manhattan
234,30990,2009-08-11,Stay in the Meatpacking District!,Manhattan,Apartment,95.000000,Entire home/apt,NaN,1.0,1,109,249,95.00000,Manhattan
368,52677,2009-11-09,Columbus Avenue Apartment,Manhattan,Apartment,NaN,Private room,NaN,1.0,1,0,190,NaN,Manhattan
426,58366,2009-11-29,"Lincoln Center luxury condo,",Manhattan,Apartment,90.000000,Entire home/apt,NaN,1.0,1,37,280,93.00000,Manhattan
464,60944,2009-12-07,Alcove Studio West Village DrmnBldg,Manhattan,Apartment,NaN,Entire home/apt,NaN,1.0,1,0,198,NaN,Manhattan
705,101172,2010-03-30,1 BR loft 5 min walk to Time Sq,Manhattan,Apartment,85.000000,Entire home/apt,NaN,3.0,1,140,150,89.00000,Manhattan
706,101172,2010-03-30,1BR w/terrace 5 min walk to Time Sq,Manhattan,Apartment,85.000000,Entire home/apt,NaN,3.0,1,78,150,86.00000,Manhattan
707,101172,2010-03-30,Luxury 1BR 5 min walk to Time Sq,Manhattan,Apartment,85.000000,Entire home/apt,NaN,3.0,1,200,150,86.00000,Manhattan
708,101172,2010-03-30,Modern 1 BR 5 min walk to Time Sq,Manhattan,Apartment,85.000000,Entire home/apt,NaN,3.0,1,79,150,89.00000,Manhattan
709,101172,2010-03-30,Unique Loft 5 min walk to Time Sq,Manhattan,Apartment,80.000000,Entire home/apt,NaN,3.0,1,35,150,84.00000,Manhattan


In [67]:
df.loc[(df['Neighbourhood'] == 'Brooklyn') & (df['Zipcode'].isna())]

,Host Id,Host Since,Name,Neighbourhood,Property Type,Review Scores Rating (bin),Room Type,Zipcode,Beds,Number of Records,Number Of Reviews,Price,Review Scores Rating,Neighbourhood
333,47716,2009-10-22,$45 L train/ 1 bedroom Brooklyn,Brooklyn,Apartment,85.000000,Private room,NaN,2.0,1,56,49,85.00000,Brooklyn
583,72808,2010-01-23,Coney island Entire Apt Safari Wi F,Brooklyn,Apartment,90.000000,Entire home/apt,NaN,2.0,1,83,105,90.00000,Brooklyn
1422,250633,2010-10-01,Historic Ditmas Park Brooklyn,Brooklyn,Apartment,100.000000,Private room,NaN,1.0,1,3,65,100.00000,Brooklyn
1930,335272,2011-01-04,Summer Green Quiet Room Park Slope,Brooklyn,House,95.000000,Private room,NaN,1.0,1,99,96,96.00000,Brooklyn
2085,389924,2011-02-13,Gorgeous French Garden cottage-N 8t,Brooklyn,Apartment,90.000000,Entire home/apt,NaN,1.0,1,38,189,92.00000,Brooklyn
2643,580566,2011-05-11,"Big bright room, great neighborhood",Brooklyn,Apartment,85.000000,Private room,NaN,1.0,1,7,60,89.00000,Brooklyn
3120,751530,2011-06-27,Room in three-Bedroom Apartment,Brooklyn,Apartment,NaN,Entire home/apt,NaN,1.0,1,0,10,NaN,Brooklyn
4013,1179779,2011-09-19,Private loft with 2 beds /bathroom,Brooklyn,Apartment,90.000000,Entire home/apt,NaN,2.0,1,43,123,92.00000,Brooklyn
4479,1349430,2011-10-29,Huge beautiful 1 bedroom loft,Brooklyn,Apartment,90.000000,Entire home/apt,NaN,2.0,1,58,140,93.00000,Brooklyn
5028,1494118,2011-12-11,Peaceful Townhouse Duplex,Brooklyn,House,100.000000,Entire home/apt,NaN,2.0,1,2,350,100.00000,Brooklyn


In [68]:
df.loc[(df['Neighbourhood'] == 'Queens') & (df['Zipcode'].isna())]

,Host Id,Host Since,Name,Neighbourhood,Property Type,Review Scores Rating (bin),Room Type,Zipcode,Beds,Number of Records,Number Of Reviews,Price,Review Scores Rating,Neighbourhood
5164,1528912,2011-12-22,Nice Private Room Beauty in Queens,Queens,Apartment,95.000000,Private room,NaN,1.0,1,23,59,95.00000,Queens
8356,3114604,2012-07-31,Beautiful room/ Double bed,Queens,Apartment,NaN,Private room,NaN,1.0,1,0,84,NaN,Queens
8576,3250450,2012-08-13,Location Marilda ( only for girls),Queens,Apartment,100.000000,Private room,NaN,1.0,1,1,43,100.00000,Queens
9510,3816303,2012-10-09,Astoria Room Close to Manhattan!!,Queens,Apartment,85.000000,Private room,NaN,1.0,1,82,56,86.00000,Queens
9718,3956850,2012-10-23,NYC Astoria Big warm apartment,Queens,Apartment,100.000000,Entire home/apt,NaN,1.0,1,2,89,100.00000,Queens
9999,4094837,2012-11-08,"BIG, SUNNY Private room n Ridgewood",Queens,Apartment,NaN,Private room,NaN,1.0,1,0,75,NaN,Queens
11111,4658655,2013-01-10,Lovely Private Room in Two Bedroom,Queens,Apartment,NaN,Private room,NaN,1.0,1,0,89,NaN,Queens
12224,5577926,2013-03-22,clean and private room in safe area,Queens,Apartment,100.000000,Private room,NaN,1.0,1,7,51,100.00000,Queens
13036,6291258,2013-05-08,"Private room in Astoria, Queens",Queens,Apartment,90.738659,Private room,NaN,1.0,1,2,65,91.99323,Queens
16201,9687105,2013-10-28,REDESIGNED LUXURY PENTHOUSE LOFT,Queens,Apartment,90.000000,Private room,NaN,1.0,1,32,135,94.00000,Queens


In [69]:
df.loc[(df['Neighbourhood'] == 'Bronx') & (df['Zipcode'].isna())]

,Host Id,Host Since,Name,Neighbourhood,Property Type,Review Scores Rating (bin),Room Type,Zipcode,Beds,Number of Records,Number Of Reviews,Price,Review Scores Rating,Neighbourhood
18911,13664245,2014-03-29,"Private, Large room with closet!",Bronx,House,90.0,Private room,NaN,4.0,1,24,48,92.0,Bronx
27217,31826196,2015-04-23,Peace and silence in New York?! Yes,Bronx,Apartment,80.0,Private room,NaN,4.0,1,6,120,80.0,Bronx


In [70]:
df.loc[(df['Neighbourhood'] == 'Staten Island') & (df['Zipcode'].isna())]

,Host Id,Host Since,Name,Neighbourhood,Property Type,Review Scores Rating (bin),Room Type,Zipcode,Beds,Number of Records,Number Of Reviews,Price,Review Scores Rating,Neighbourhood
11693,5133304,2013-02-18,Your home in NYC close to Manhattan,Staten Island,House,90.0,Entire home/apt,NaN,2.0,1,24,80,91.0,Staten Island


In [71]:
print(NaN_count)

Host Id                         0
Host Since                      2
Name                            0
Neighbourhood                   0
Property Type                   2
Review Scores Rating (bin)      0
Room Type                       0
Zipcode                       104
Beds                           46
Number of Records               0
Number Of Reviews               0
Price                           0
Review Scores Rating            0
dtype: int64


In [73]:
df.to_csv('airbnb.csv' ,index=False)